<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b2b652d5c1a9514ad5248e2993eead5ed8763adae5b1cfd327f3d9cc5a3c1002
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-24 13:17:34
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.40 C
-------------------
Today PnL: -36.28 K (-0.26%)
Current PnL: -19.30 L (-12.98%)
CY Booked + Current PnL: -7.86 L (-5.28%)
-------------------
Total profit:  1.52 L
Total loss:  -20.81 L
-------------------
Total Booked + Current PnL: 19.12 L (15.71%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.15%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 85.16 L (60.7%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.7%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,12.54,57.0,H-MC,3.77,194220.0,29550.0,13790.0,-1.11,17.94,7.10,26.31,89.0,2.14,1.38,27.36,X40N,NTT,AC
77,TTKPRESTIG,770.00,96.37,39.0,M-SC,2.93,84646.0,-16131.0,16227.0,-0.49,-16.01,19.17,0.09,245.0,-0.99,0.60,11.21,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-44.49,61.0,H-MC,9.92,111268.0,10764.0,18348.0,0.53,10.71,16.49,28.96,79.0,0.59,0.79,48.32,MH,ATH,METALS
57,RELIANCE,1533.00,-13.29,50.0,H-LC,4.46,217308.0,6942.0,21839.0,0.23,3.30,10.05,13.68,37.0,0.32,1.55,20.37,XY25,NTT,REFINERIES
84,VOLTAS,1530.00,-1.50,47.0,H-MC,2.98,206550.0,14808.0,22948.0,0.31,7.72,11.11,19.69,99.0,0.65,1.47,15.05,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-41.65,55.0,L-SC,42.91,87360.0,-26189.0,66236.0,2.22,-23.06,75.82,35.27,268.0,-0.40,0.62,113.82,XR,NTT,HOTELS
31,HINDUNILVR,2922.00,-8.38,48.0,H-LC,10.03,257410.0,347.0,34802.0,2.06,0.13,13.52,13.67,24.0,0.01,1.84,20.57,XY25,NTT,FMCG
71,TANLA,1963.11,-14.58,73.0,H-SC,15.46,228964.0,-41797.0,371746.0,1.60,-15.44,162.36,121.86,133.0,-0.11,1.63,79.11,AR,ATH,IT
68,STARHEALTH,761.00,11.71,61.0,H-SC,4.85,249043.0,-32080.0,164941.0,1.54,-11.41,66.23,47.26,144.0,-0.19,1.78,33.43,XY24,NTT,INSURANCE
69,SURYODAY,240.00,51.32,65.0,H-SC,1.07,138261.0,-40810.0,106544.0,1.49,-22.79,77.06,36.71,135.0,-0.38,0.99,37.18,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-66.09,44.0,H-MC,3.42,241710.0,-60041.0,221455.0,-2.69,-19.90,91.62,53.50,116.0,-0.27,1.72,7.36,XY24,NTT,FMCG
7,ATULAUTO,844.0,3748.65,55.0,M-SC,5.97,120053.0,-27315.0,79127.0,-2.29,-18.54,65.91,35.16,236.0,-0.35,0.86,22.62,XY24,NTT,AUTO
26,GREENPANEL,537.0,210.72,48.0,M-SC,0.79,138787.0,-44291.0,123271.0,-2.20,-24.19,88.82,43.14,230.0,-0.36,0.99,30.55,XY24,NTT,MISC
16,CAMPUS,393.0,-26.15,47.0,M-SC,0.39,153886.0,-13889.0,69726.0,-2.15,-8.28,45.31,33.28,210.0,-0.20,1.10,19.27,XY24,NTT,FOOTWEAR
18,CERA,9475.0,-23.63,31.0,H-SC,2.18,139138.0,-36765.0,78780.0,-2.11,-20.90,56.62,23.89,149.0,-0.47,0.99,20.16,OX40N,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1234.93,75.0,M-SC,9.93,181524.0,381.0,45091.0,-1.83,0.21,24.84,25.10,235.0,0.01,1.29,45.47,XY24,NTT,HEALTHCARE
35,ICICIPRULI,790.00,-21.34,43.0,H-MC,2.01,135261.0,-527.0,43284.0,-0.39,-0.39,32.00,31.48,107.0,-0.01,0.96,11.68,X40,ATH,INSURANCE
52,PGHH,17973.08,-30.02,54.0,H-MC,3.67,202425.0,1605.0,67165.0,0.65,0.80,33.18,34.25,80.0,0.02,1.44,6.04,X40,ATH,FMCG
86,WIPRO,420.00,-13.34,42.0,M-LC,5.38,151993.0,1048.0,108401.0,-1.79,0.69,71.32,72.51,53.0,0.01,1.08,6.76,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,93.70,51.0,H-SC,9.64,122175.0,-20304.0,39829.0,0.09,-14.25,32.60,13.70,163.0,-0.51,0.87,44.20,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,96.37,39.0,M-SC,2.93,84646.0,-16131.0,16227.0,-0.49,-16.01,19.17,0.09,245.0,-0.99,0.60,11.21,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-68.16,48.0,H-SC,1.09,219564.0,-50103.0,86442.0,-0.91,-18.58,39.37,13.47,138.0,-0.58,1.57,11.05,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-22.20,43.0,H-MC,6.89,105203.0,-26632.0,68855.0,-0.33,-20.20,65.45,32.03,98.0,-0.39,0.75,17.84,OX40N,NTT,IT
66,SIS,528.0,2078.76,37.0,H-SC,4.67,87937.0,-23095.0,46703.0,-0.10,-20.80,53.11,21.26,156.0,-0.49,0.63,18.81,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1234.93,75.0,M-SC,9.93,181524.0,381.0,45091.0,-1.83,0.21,24.84,25.10,235.0,0.01,1.29,45.47,XY24,NTT,HEALTHCARE
60,SAIL,228.00,49.96,71.0,M-MC,9.77,236723.0,11761.0,155882.0,0.12,5.23,65.85,74.53,192.0,0.08,1.69,39.17,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,12.54,57.0,H-MC,3.77,194220.0,29550.0,13790.0,-1.11,17.94,7.10,26.31,89.0,2.14,1.38,27.36,X40N,NTT,AC
84,VOLTAS,1530.00,-1.50,47.0,H-MC,2.98,206550.0,14808.0,22948.0,0.31,7.72,11.11,19.69,99.0,0.65,1.47,15.05,XY25,NTT,AC
17,CAMS,5211.76,-3.17,52.0,H-SC,2.25,110110.0,8106.0,35819.0,-1.34,7.95,32.53,43.06,122.0,0.23,0.79,26.42,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-44.49,61.0,H-MC,9.92,111268.0,10764.0,18348.0,0.53,10.71,16.49,28.96,79.0,0.59,0.79,48.32,MH,ATH,METALS
11,BANKINDIA,190.00,-26.71,72.0,H-MC,7.50,189020.0,9212.0,103205.0,1.21,5.12,54.60,62.52,88.0,0.09,1.35,40.96,XR,NTT,BANKS
85,WHIRLPOOL,2270.00,-42.31,42.0,M-SC,1.42,97582.0,6084.0,72669.0,-1.50,6.65,74.47,86.07,223.0,0.08,0.70,41.71,XR,NTT,DURABLES
37,INDIAMART,4850.92,-54.64,27.0,H-SC,10.72,126516.0,3180.0,130577.0,-0.16,2.58,103.21,108.45,119.0,0.02,0.90,25.51,AR,ATH,MISC
86,WIPRO,420.00,-13.34,42.0,M-LC,5.38,151993.0,1048.0,108401.0,-1.79,0.69,71.32,72.51,53.0,0.01,1.08,6.76,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-19.59,33.0,H-LC,7.71,289812.0,-26030.0,138704.0,-0.93,-8.24,47.86,35.67,5.0,-0.19,2.07,4.47,X40N,ATH,FMCG
42,ITC,452.00,-39.19,39.0,H-LC,2.07,196578.0,-3560.0,24454.0,-0.30,-1.78,12.44,10.44,4.0,-0.15,1.40,3.69,X40,NTT,FMCG
44,JIOFIN,387.00,-1.88,41.0,H-LC,13.27,266643.0,-722.0,66181.0,-0.80,-0.27,24.82,24.48,48.0,-0.01,1.90,54.55,XY24,BTT,FINANCE
30,HCLTECH,1943.91,-0.65,43.0,H-LC,9.91,226410.0,-15514.0,78791.0,0.86,-6.41,34.80,26.15,8.0,-0.20,1.61,6.99,X40,ATH,IT
74,TCS,4476.75,-27.72,43.0,H-LC,12.16,287866.0,-58090.0,132937.0,-0.37,-16.79,46.18,21.64,1.0,-0.44,2.05,1.98,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-5.49,46.0,H-LC,1.53,234292.0,-11859.0,91069.0,-1.06,-4.82,38.87,32.18,16.0,-0.13,1.67,29.49,X200,ATH,IT
42,ITC,452.0,-39.19,39.0,H-LC,2.07,196578.0,-3560.0,24454.0,-0.30,-1.78,12.44,10.44,4.0,-0.15,1.40,3.69,X40,NTT,FMCG
65,SIEMENS,4671.5,0.17,52.0,H-LC,3.07,160895.0,-25200.0,72676.0,-0.69,-13.54,45.17,25.51,15.0,-0.35,1.15,18.57,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-13.29,50.0,H-LC,4.46,217308.0,6942.0,21839.0,0.23,3.30,10.05,13.68,37.0,0.32,1.55,20.37,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-10.01,49.0,H-LC,4.54,276799.0,11373.0,45423.0,1.24,4.28,16.41,21.40,11.0,0.25,1.97,11.68,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-27.72,43.0,H-LC,12.16,287866.0,-58090.0,132937.0,-0.37,-16.79,46.18,21.64,1.0,-0.44,2.05,1.98,X40,ATH,IT
42,ITC,452.00,-39.19,39.0,H-LC,2.07,196578.0,-3560.0,24454.0,-0.30,-1.78,12.44,10.44,4.0,-0.15,1.40,3.69,X40,NTT,FMCG
83,VBL,671.64,-19.59,33.0,H-LC,7.71,289812.0,-26030.0,138704.0,-0.93,-8.24,47.86,35.67,5.0,-0.19,2.07,4.47,X40N,ATH,FMCG
80,UNITDSPR,1644.00,-11.23,60.0,H-LC,7.25,232871.0,-1273.0,49904.0,0.95,-0.54,21.43,20.77,86.0,-0.03,1.66,6.34,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-0.65,43.0,H-LC,9.91,226410.0,-15514.0,78791.0,0.86,-6.41,34.80,26.15,8.0,-0.20,1.61,6.99,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.17,52.0,H-LC,3.07,160895.0,-25200.0,72676.0,-0.69,-13.54,45.17,25.51,15.0,-0.35,1.15,18.57,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,39.0,H-LC,2.07,196578.0,-3560.0,24454.0,-0.30,-1.78,12.44,10.44,4.0,-0.15,1.40,3.69,X40,NTT,FMCG
19,CIPLA,1795.00,-21.15,45.0,H-LC,5.17,210843.0,6343.0,35063.0,0.78,3.10,16.63,20.25,10.0,0.18,1.50,12.11,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-13.98,44.0,H-LC,7.58,217272.0,-34496.0,87713.0,0.90,-13.70,40.37,21.13,27.0,-0.39,1.55,16.87,X40,ATH,PAINTS
57,RELIANCE,1533.00,-13.29,50.0,H-LC,4.46,217308.0,6942.0,21839.0,0.23,3.30,10.05,13.68,37.0,0.32,1.55,20.37,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-27.72,43.0,H-LC,12.16,287866.0,-58090.0,132937.0,-0.37,-16.79,46.18,21.64,1.0,-0.44,2.05,1.98,X40,ATH,IT
40,INFY,2275.00,-18.52,47.0,H-LC,8.13,317306.0,4418.0,167284.0,-0.52,1.41,52.72,54.87,3.0,0.03,2.26,8.16,X40,BTT,IT
42,ITC,452.00,-39.19,39.0,H-LC,2.07,196578.0,-3560.0,24454.0,-0.30,-1.78,12.44,10.44,4.0,-0.15,1.40,3.69,X40,NTT,FMCG
83,VBL,671.64,-19.59,33.0,H-LC,7.71,289812.0,-26030.0,138704.0,-0.93,-8.24,47.86,35.67,5.0,-0.19,2.07,4.47,X40N,ATH,FMCG
1,ABB,7934.00,-39.28,52.0,H-LC,8.63,253622.0,-7997.0,127217.0,-0.28,-3.06,50.16,45.57,7.0,-0.06,1.81,7.92,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6883.33,56.0,L-SC,5.35,76630.0,-17180.0,93795.0,-0.90,-18.31,122.40,81.67,269.0,-0.18,0.55,48.40,XR,NTT,CERAMICS
61,SAMMAANCAP,326.00,-171.59,56.0,M-SC,4.90,82140.0,-20070.0,113460.0,1.39,-19.64,138.13,91.37,208.0,-0.18,0.59,33.34,XY25,NTT,FINANCE
21,COFFEEDAY,80.00,-41.65,55.0,L-SC,42.91,87360.0,-26189.0,66236.0,2.22,-23.06,75.82,35.27,268.0,-0.40,0.62,113.82,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-6.52,42.0,H-SC,11.75,87075.0,-13888.0,108417.0,0.03,-13.76,124.51,93.63,148.0,-0.13,0.62,29.70,SR,ATH,CHEMICALS
49,MASFIN,398.61,-16.44,55.0,H-SC,12.70,95190.0,-2790.0,24397.0,0.92,-2.85,25.63,22.05,152.0,-0.11,0.68,38.37,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1234.93,75.0,M-SC,9.93,181524.0,381.0,45091.0,-1.83,0.21,24.84,25.10,235.0,0.01,1.29,45.47,XY24,NTT,HEALTHCARE
71,TANLA,1963.11,-14.58,73.0,H-SC,15.46,228964.0,-41797.0,371746.0,1.60,-15.44,162.36,121.86,133.0,-0.11,1.63,79.11,AR,ATH,IT
7,ATULAUTO,844.00,3748.65,55.0,M-SC,5.97,120053.0,-27315.0,79127.0,-2.29,-18.54,65.91,35.16,236.0,-0.35,0.86,22.62,XY24,NTT,AUTO
81,VAIBHAVGBL,521.00,59.78,59.0,H-SC,5.97,138383.0,-44392.0,155459.0,-0.73,-24.29,112.34,60.77,125.0,-0.29,0.99,23.94,XR,NTT,JEWELLERY
60,SAIL,228.00,49.96,71.0,M-MC,9.77,236723.0,11761.0,155882.0,0.12,5.23,65.85,74.53,192.0,0.08,1.69,39.17,XY24,BTT,STEEL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.26
1,25,43.70
2,50,74.75


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.02
LC    32.45
MC    23.52
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.26
X40      14.71
X40N     11.39
XR       11.32
XY25     10.12
AR        8.43
OX40N     7.81
X200      1.67
MH        1.63
X5K       1.42
SR        1.23
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.39
H-LC    25.99
H-MC    20.56
M-SC    13.10
M-LC     5.42
M-MC     2.66
L-SC     1.53
L-LC     1.04
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.90,-5.24,39.98
IT,12.97,-15.47,78.43
FINANCE,10.05,-16.38,67.16
MISC,7.02,-16.95,81.97
BANKS,6.18,-15.15,75.72
PAINTS,5.63,-16.91,34.34
ELECTRICAL,5.47,-7.36,45.03
INSURANCE,3.73,-5.71,45.22
AC,3.55,2.88,13.38


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3136016.0,22
XR,1323878.0,14
AR,1150805.0,9
X40,783428.0,10
X40N,606723.0,9
OX40N,551630.0,10
XY25,420353.0,7
SR,249219.0,2
X5K,118920.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3673337.0,29
M-SC,1410632.0,17
H-MC,1218900.0,15
H-LC,1174057.0,15
M-LC,390102.0,4
M-MC,304531.0,2
L-SC,246287.0,3
L-MC,60028.0,1
L-LC,38471.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1233540.0      6
M-SC       XY24       805655.0      7
H-SC       AR         785634.0      5
           XR         774441.0      7
H-MC       XY24       563328.0      4
H-LC       X40        491179.0      5
M-MC       XY24       304531.0      2
H-SC       X40N       303490.0      4
           OX40N      261057.0      4
           SR         249219.0      2
H-LC       X40N       223671.0      3
H-MC       X40        219244.0      4
H-LC       AR         199893.0      2
H-MC       XY25       166358.0      2
M-SC       AR         165278.0      2
M-LC       XY24       162781.0      2
L-SC       XR         160031.0      2
M-SC       OX40N      135462.0      4
M-LC       X5K        118920.0      1
M-SC       XR         117772.0      2
           XY25       113460.0      1
M-LC       XR         108401.0      1
H-MC       XR         103205.0      1
H-LC       XY25       102064.0      3
           X200        91069.0      1
L-SC       OX40N       86256.0      1
H-MC       X40N        79562.0      2
M-SC       X40         73005.0      1
H-MC       OX40N       68855.0      1
H-LC       XY24        66181.0      1
H-SC       MH          65956.0      1
L-MC       XR          60028.0      1
L-LC       XY25        38471.0      1
H-MC       MH          18348.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
